In [1]:
import models.MLP.model as MLP
import models.MLP.experiments as experiments
import models.CNN.model as CNN
import models.MLP.layers as layers_file
import importlib
import numpy as np
from tqdm import tqdm
import torch
from models.MLP.generate_configs import generate_mlp_config_files
import json

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from tqdm.auto import tqdm
import itertools
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import models.learning as L

transform_MNIST = transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.view(-1))
        ])
train_dataset = datasets.MNIST(root='./data', train=True, download=False, transform=transform_MNIST)
test_dataset = datasets.MNIST(root='./data', train=False, download=False, transform=transform_MNIST)

train_dataloader = DataLoader(train_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

/Users/zhaoyizhe/miniconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/zhaoyizhe/miniconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/zhaoyizhe/miniconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <539E3895-EEBD-3044-801F-9B618F314545> /Users/zhaoyizhe/miniconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [20]:
import importlib
import itertools

In [37]:
importlib.reload(MLP)
importlib.reload(L)

<module 'models.learning' from '/Users/zhaoyizhe/Documents/GitHub/biological-deep-learning/models/learning.py'>

In [2]:
generate_mlp_config_files(hsizes=[64], lambds=[5], wlrs=[0.01], blrs=[0.01], llrs=[0.01])

In [6]:
device = torch.device('cpu')

In [3]:
with open("ConfigsMLP/config0.json", "r") as file:
    config = json.load(file)

In [7]:
model = MLP.NewMLPBaseline_Model(config["hsize"], config["lambd"], config["w_lr"], config["b_lr"], config["l_lr"], nclasses=10, device=device)
# Set greedytrain = true for layer wise training 
model = experiments.SoftMLPBaseline_Experiment(1, model, train_dataloader, 'MNIST', 10, device, greedytrain=False)

  0%|          | 0/60000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
print("\nTesting the model...")
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for testing
    for data in test_dataloader:  # Loop over the test set
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model.forward(inputs)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")


Testing the model...
Accuracy on the test set: 9.80%
